# Deuxième partie : Détection des tumeurs cérébrales avec YOLOv8

Cette partie consiste à classer les images de tumeurs cérébrales et à localiser les tumeurs à l'aide du modèle YOLOv8.

In [ ]:
# Imports et configuration initiale

## 1. Afficher un échantillon d'images pour chaque classe avec les boîtes englobantes

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Dossier racine du dataset
data_dir = "../Data/Raw/Data_Brain/Train"

# Récupérer les noms des classes (dossiers)
classes = os.listdir(data_dir)
print("Classes :", classes)

# Fonction pour convertir les coordonnées YOLO en coordonnées d'image
def yolo_to_box(x_center, y_center, width, height, img_width, img_height):
    x_min = (x_center - width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    box_width = width * img_width
    box_height = height * img_height
    return x_min, y_min, box_width, box_height

# Afficher une image pour chaque classe
plt.figure(figsize=(15, 5))

for i, cls in enumerate(classes):
    img_dir = os.path.join(data_dir, cls, "images")
    label_dir = os.path.join(data_dir, cls, "labels")

    # Prendre la première image et son label correspondant
    img_name = os.listdir(img_dir)[0]
    label_name = os.path.splitext(img_name)[0] + ".txt"

    img_path = os.path.join(img_dir, img_name)
    label_path = os.path.join(label_dir, label_name)

    # Ouvrir l’image
    img = Image.open(img_path).convert("RGB")
    img_w, img_h = img.size

    # Lire le fichier d’annotation
    with open(label_path, "r") as f:
        lines = f.readlines()

    # Afficher l’image
    ax = plt.subplot(1, len(classes), i + 1)
    ax.imshow(img)
    ax.set_title(cls)
    ax.axis("off")

    # Dessiner les boîtes englobantes
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            continue
        _, x, y, w, h = map(float, parts)
        x_min, y_min, bw, bh = yolo_to_box(x, y, w, h, img_w, img_h)
        rect = patches.Rectangle(
            (x_min, y_min), bw, bh, linewidth=2, edgecolor='red', facecolor='none'
        )
        ax.add_patch(rect)

plt.tight_layout()
plt.show()


## 3. Filtrer et copier les images et labels

Pour chaque image :
- Vérifier si un fichier .txt correspondant existe dans le répertoire des labels
- Si un label est trouvé : copier l'image et le label vers le dossier approprié (train/valid/test)
- Si aucun label n'est trouvé : afficher un avertissement et sauter l'image

In [ ]:
import os
import shutil

# === 1 Dossiers source et sortie ===
source_dir = "../Data/Raw/Data_Brain"   # Ton dataset
output_dir = "../Data/outputpath"      # Nouveau dossier de sortie

# Créer le dossier de sortie et ses sous-dossiers
for sub in ["images", "labels"]:
    for split in ["Train", "Val"]:
        path = os.path.join(output_dir, sub, split)
        os.makedirs(path, exist_ok=True)

print(" Dossiers créés avec succès !")

# === 2 Extensions d’images acceptées ===
valid_ext = (".jpg", ".jpeg", ".png", ".bmp")

# === 3 Parcourir Train et Val ===
for split in ["Train", "Val"]:
    split_path = os.path.join(source_dir, split)

    print(f"\n🔹 Traitement de {split_path}")

    # Parcourir les classes (Glioma, Meningioma, etc.)
    for cls in os.listdir(split_path):
        class_path = os.path.join(split_path, cls)
        if not os.path.isdir(class_path):
            continue

        img_folder = os.path.join(class_path, "images")
        label_folder = os.path.join(class_path, "labels")

        if not os.path.exists(img_folder) or not os.path.exists(label_folder):
            print(f"⚠️ {cls} n’a pas de dossier images ou labels.")
            continue

        for img_name in os.listdir(img_folder):
            if not img_name.lower().endswith(valid_ext):
                continue

            label_name = os.path.splitext(img_name)[0] + ".txt"
            img_path = os.path.join(img_folder, img_name)
            label_path = os.path.join(label_folder, label_name)

            if os.path.exists(label_path):
                shutil.copy(img_path, os.path.join(output_dir, "images", split, img_name))
                shutil.copy(label_path, os.path.join(output_dir, "labels", split, label_name))
            else:
                print(f"⚠️ Pas de label pour {img_name}")

print("\n✅ Copie terminée avec succès ! Seules les images avec labels ont été copiées.")


## 4. Créer le fichier data.yaml sans augmentations

Ce fichier contient :
- Les chemins des dossiers d'entraînement, de validation et de test
- Le nombre de classes et le nom de chaque classe
- Configuration sans augmentations de données

In [ ]:
import yaml

data_config = {
    'train': 'Data/outputpath/images/Train',
    'val': 'Data/outputpath/images/Val',
    'nc': 4,
    'names': ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary'],
    'augment': False
}

with open('../data.yaml', 'w') as file:
    yaml.dump(data_config, file)

print("✅ Fichier data2.yaml créé avec succès (avec augmentations desactivées) !")


## 5. Créer le fichier data2.yaml avec augmentations

Ce fichier contient la même structure que data.yaml mais avec des augmentations de données activées

In [ ]:
import yaml

data2_config = {
    'train': 'Data/outputpath/images/Train',
    'val': 'Data/outputpath/images/Val',
    'nc': 4,
    'names': ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary'],
    'augment': True
}

with open('../data2.yaml', 'w') as file:
    yaml.dump(data2_config, file)

print("✅ Fichier data2.yaml créé avec succès (avec augmentations activées) !")


## 6. Compter le nombre d'images et d'étiquettes

Compter les images et labels présents dans les ensembles d'entraînement et de validation

## 7. Vérifier la correspondance images-labels et nettoyer les données

- Vérifier que chaque image possède un label correspondant
- Supprimer les images sans label
- Supprimer les labels sans image correspondante

## 8. Entraîner le modèle YOLOv8

Lancer l'entraînement du modèle YOLO en définissant les hyperparamètres appropriés

## 9. Évaluer et tester le modèle

Évaluer et tester le modèle après l'entraînement pour mesurer :
- Les performances
- La précision
- La capacité à généraliser sur des données inédites

## 10. Sauvegarder le modèle entraîné

Sauvegarder le modèle entraîné pour une utilisation future